In [1]:
# libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline


from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.tree import plot_tree, export_text

from pprint import pprint

## Setup

In [2]:
#read file
uavsar_pits = pd.read_csv('/home/naheemadebisi/PhD/snow-analytics/Radar-Backscatter/pits_data/uavsar_SWE_ENV.csv')
#drop null
uavsar_pits.dropna(inplace = True)
uavsar_pits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 261 entries, 0 to 285
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             261 non-null    int64  
 1   Location               261 non-null    object 
 2   PitID                  261 non-null    object 
 3   Density Mean (kg/m^3)  261 non-null    float64
 4   Snow Depth (cm)        261 non-null    float64
 5   SWE (mm)               261 non-null    float64
 6   HH                     261 non-null    float64
 7   HV                     261 non-null    float64
 8   VH                     261 non-null    float64
 9   VV                     261 non-null    float64
 10  Temp_mean              261 non-null    float64
 11  Ground Condition       261 non-null    object 
 12  Ground Roughness       261 non-null    object 
 13  Ground Vegetation      261 non-null    object 
 14  Canopy                 261 non-null    object 
dtypes: flo

In [3]:
#train-test split
train_df , test_df = train_test_split(uavsar_pits, test_size= 0.25, random_state= 20221104)
print(f"lenth of the training data is {len(train_df)}")
print(f"lenth of the testdata is {len(test_df)}")

lenth of the training data is 195
lenth of the testdata is 66


In [4]:
#input and output data
input_col = ['Density Mean (kg/m^3)', 'Snow Depth (cm)', 'SWE (mm)', 'Temp_mean',
       'Ground Condition', 'Ground Roughness', 'Canopy']

train_X = train_df[input_col].copy()
test_X = test_df[input_col].copy()

out_col = 'HH'
train_Y_HH = train_df[out_col].copy()
test_Y_HH = test_df[out_col].copy()


#select the four numerical cols(Density, Depth, SWE, temp)
num_cols = train_X.select_dtypes(include = np.number).columns.tolist()
#Select the four categorical cols (Ground Condition, Ground Roughness, Ground Vegetation, Canopy)
cat_cols = train_X.select_dtypes('object').columns.tolist()

In [5]:
#let's see some stats for the numerical columns
train_X[num_cols].describe()

,Density Mean (kg/m^3),Snow Depth (cm),SWE (mm),Temp_mean
count,195.000000,195.000000,195.000000,195.000000
mean,250.743590,113.123077,299.448718,-4.112674
std,72.657319,55.229304,203.338122,2.678310
min,75.000000,22.000000,55.000000,-16.250000
25%,205.250000,73.500000,134.000000,-5.506250
50%,268.000000,106.000000,260.000000,-3.831250
75%,299.000000,135.000000,389.750000,-2.151429
max,436.500000,284.000000,969.000000,0.000000


In [6]:
#let's check the categories in each category
train_X[cat_cols].nunique()

Ground Condition    3
Ground Roughness    3
Canopy              4
dtype: int64

## Modelling

Todo: redo steps following jovian!

In [7]:
#initiate the columnTransformer
column_trans = ColumnTransformer((
                ('categories', OneHotEncoder(), make_column_selector(dtype_include= object)),
                ('scaler', StandardScaler(), make_column_selector(dtype_include= np.number))
                      ))

In [8]:
#Put transformer and predictor in a pipeline
rf = Pipeline([
            ('features', column_trans),
            ('pred', RandomForestRegressor())
            ])
param_grid = dict(pred__n_estimators = [10, 30, 90, 270, 500, 1000], pred__max_depth = [1, 3, 9, 21, 63, 189, 250], 
                pred__max_leaf_nodes = [1, 5, 25, 75, 135, 175, 200, None],  pred__max_features = [ None, 'sqrt', 'log2'],
                  pred__min_samples_split = [1, 2, 4, 8, 16, 32],  pred__min_samples_leaf = [1, 2, 4, 8, 16, 32], pred__bootstrap = [True, False])

rf_gs = GridSearchCV(rf, param_grid= param_grid, n_jobs = -1)

In [9]:
rf_gs.fit(train_X, train_Y_HH)

/home/naheemadebisi/miniconda3/envs/earth-analytics-python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
98280 fits failed out of a total of 362880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30240 fits failed with the following error:
Traceback (most recent call last):
  File "/home/naheemadebisi/miniconda3/envs/earth-analytics-python/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/naheemadebisi/miniconda3/envs/earth-analytics-python/lib/python3.8/site-packages/sklearn/pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/nah

GridSearchCV(estimator=Pipeline(steps=[('features',
                                        ColumnTransformer(transformers=(('categories',
                                                                         OneHotEncoder(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f8be73565b0>),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x7f8be73566d0>)))),
                                       ('pred', RandomForestRegressor())]),
             n_jobs=-1,
             param_grid={'pred__bootstrap': [True, False],
                         'pred__max_depth': [1, 3, 9, 21, 63, 189, 250],
                         'pred__max_features': [None, 'sqrt', 'log2'],
                         'pred__max_leaf_nodes': [1, 5, 25, 75, 135, 175, 200,
                                                  None],
                         'pred__min_samples_leaf': [1, 2, 4, 8, 16, 32],
                         'pred__min_samples_split': [1, 2, 4, 8, 16, 32],
                         'pred__n_estimators': [10, 30, 90, 270, 500, 1000]})

In [15]:
#predict train
predictions_train = rf_gs.best_estimator_.predict(train_X)
#predict test
predictions_test = rf_gs.best_estimator_.predict(test_X)

#compute loss train
mse_loss = mse(train_Y_HH, predictions_train)
rsquare = r2_score(train_Y_HH, predictions_train) 
print(f'Train mse: {mse_loss}')
print(f'Train r_square: {rsquare}')

#compute loss test
mse_loss = mse(test_Y_HH, predictions_test)
rsquare = r2_score(test_Y_HH, predictions_test) 
print(f'Test mse: {mse_loss}')
print(f'Test r_square: {rsquare}')

Train mse: 0.007753746195324136
Train r_square: 0.37575065325496515
Test mse: 0.00997885213795148
Test r_square: 0.35819428270943354


In [16]:
predictions_test

array([0.12933889, 0.27202738, 0.27202738, 0.12933889, 0.27202738,
       0.12933889, 0.27202738, 0.27202738, 0.27202738, 0.12933889,
       0.12933889, 0.27202738, 0.12933889, 0.27202738, 0.12933889,
       0.27202738, 0.12933889, 0.27202738, 0.27202738, 0.27202738,
       0.27202738, 0.12933889, 0.12933889, 0.27202738, 0.27202738,
       0.12933889, 0.12933889, 0.27202738, 0.27202738, 0.12933889,
       0.27202738, 0.27202738, 0.12933889, 0.12933889, 0.12933889,
       0.27202738, 0.12933889, 0.27202738, 0.27202738, 0.12933889,
       0.12933889, 0.12933889, 0.12933889, 0.27202738, 0.27202738,
       0.27202738, 0.27202738, 0.27202738, 0.12933889, 0.12933889,
       0.27202738, 0.27202738, 0.12933889, 0.27202738, 0.27202738,
       0.27202738, 0.27202738, 0.27202738, 0.12933889, 0.12933889,
       0.12933889, 0.12933889, 0.27202738, 0.27202738, 0.27202738,
       0.27202738])

In [20]:
def feature_importance(fitted_model):
    #feature importance
    importance_df = pd.DataFrame({
    'feature': uavsar_pits[input_col].columns,
    'importance': fitted_model.named_steps["pred"].feature_importances_
    }).sort_values('importance', ascending=False)

    plt.title('Feature Importance')
    sns.barplot(data=importance_df.head(10), x='importance', y='feature');

In [28]:
rf_gs.best_estimator_.named_steps["pred"].feature_importances_

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [21]:
feature_importance(rf_gs.best_estimator_)

ValueError: All arrays must be of the same length